In [1]:
# Imports
!pip install cpmpy numpy --quiet

import cpmpy as cp
import numpy as np

(You can ignore google* and tensor* dependency errors on Google Colab.)


## **Session 1: Introduction to CPMpy and Basic Modelling**

In this session, we will be introducing you to constraint programming using the CPMpy framework and working through some modelling exercises. The goal is to help you become familiar with how to formulate/model combinatorial problems using CPMpy.

<!-- <strong>Plan for the session:</strong>
- Introductory Talk (~20 minutes): Introduction to CPMpy, constraint programming, and how combinatorial problems can be represented as models.
- Hands-on Exercises (~2 hours): Practice modelling problems using CPMpy through a series of guided exercises. -->


<strong>If in doubt, quickly check the summary sheet:</strong>
* CPMpy summary sheet: https://cpmpy.readthedocs.io/en/latest/summary.html

<strong>If things are unclear, check the extensive modeling guide:</strong>

* CPMpy modeling documentation: https://cpmpy.readthedocs.io/en/latest/modeling.html

### **Part 1: Short Introduction**

#### **What is Constraint Programming (CP)?**

Constraint Programming is a **declarative paradigm** for solving **combinatorial problems**. In CP, a problem is modelled by defining:

- **Decision Variables**: They represent the solution, as entities that their values need to be determined.
- **Domains**: The set of possible values for the decision variables.
- **Constraints**: Relationships between variables restricting their possible values.
- **Objective (Optional)**: If the problem includes an optimisation objective, we can define an objective to minimize or maximize.

CP allows you to focus on **what** needs to be satisfied, without specifying **how** to solve the problem.

#### **Imperative vs Declarative Programming**

To understand the difference between imperative and declarative programming, let's take a simple example:

We have two variables `x` and `y`, and we want to find their values such that:
- Their sum is equal to 7.
- `x` is greater than `y`.

In [2]:
# Imperative
x, y = 0, 0

print(f"Imperative Approach: We explicitly assign the variables to specific values: x={x} and y={y}")

print("... and we explicitly reassign it until we find the solution:")
found = False
for x in range(10):
    for y in range(10):
        print(f"x = {x}, y = {y}")
        if x + y == 7 and x > y:
            print(f"\nSolution found: x = {x}, y = {y}")
            found = True
            break
    if found:
        break

Imperative Approach: We explicitly assign the variables to specific values: x=0 and y=0
... and we explicitly reassign it until we find the solution:
x = 0, y = 0
x = 0, y = 1
x = 0, y = 2
x = 0, y = 3
x = 0, y = 4
x = 0, y = 5
x = 0, y = 6
x = 0, y = 7
x = 0, y = 8
x = 0, y = 9
x = 1, y = 0
x = 1, y = 1
x = 1, y = 2
x = 1, y = 3
x = 1, y = 4
x = 1, y = 5
x = 1, y = 6
x = 1, y = 7
x = 1, y = 8
x = 1, y = 9
x = 2, y = 0
x = 2, y = 1
x = 2, y = 2
x = 2, y = 3
x = 2, y = 4
x = 2, y = 5
x = 2, y = 6
x = 2, y = 7
x = 2, y = 8
x = 2, y = 9
x = 3, y = 0
x = 3, y = 1
x = 3, y = 2
x = 3, y = 3
x = 3, y = 4
x = 3, y = 5
x = 3, y = 6
x = 3, y = 7
x = 3, y = 8
x = 3, y = 9
x = 4, y = 0
x = 4, y = 1
x = 4, y = 2
x = 4, y = 3

Solution found: x = 4, y = 3


In [3]:
# Declarative

# Define decision variables with domains
x, y = cp.intvar(0, 10, shape=2)

print(f"Declarative Approach: We define the variables x={x} and y={y}.")
print("Notice that we don't explicitly set/assign any values to the variables. We simply define their domains (sets of possible values).")

print("\n... and we model the problem declaratively (i.e. we do not specify how to find the correct assignment, but what the correct assignment should satisfy):\n")

# Model the constraints
model = cp.Model([
    x + y == 7,  # Their sum is 7
    x > y         # x is greater than y
])

print(model)

# Solve the model
print("\n... and we solve the problem:\n")
model.solve()

print(f"Solution: x = {x.value()}, y = {y.value()}")

Declarative Approach: We define the variables x=IV0 and y=IV1.
Notice that we don't explicitly set/assign any values to the variables. We simply define their domains (sets of possible values).

... and we model the problem declaratively (i.e. we do not specify how to find the correct assignment, but what the correct assignment should satisfy):

Constraints:
    (IV0) + (IV1) == 7
    (IV0) > (IV1)
Objective: None

... and we solve the problem:

Solution: x = 4, y = 3


#### **What is CPMpy?**
CPMpy is a Constraint Programming library in Python, for **modelling** and **solving** combinatorial problems.

**Basic Model Structure**:
1. **Decision Variables**: Defined with `intvar()` for integers or `boolvar()` for booleans.
2. **Constraints**: Applied on variables using comparison operators (`==`, `!=`, `<`, `<=`, etc.) or [global constraints](https://cpmpy.readthedocs.io/en/latest/modeling.html#global-constraints) like `AllDifferent`.
3. **Solving**: Use `solve()` to find a solution.

#### **A Practical Example**

We need to solve a Sudoku puzzle. The goal of Sudoku is to fill a 9x9 grid with numbers so that each row, column and 3x3 section contain all the digits
between 1 and 9.
The given unsolved grid is the following:

```
[  
    [0, 0, 0,  2, 0, 5,  0, 0, 0],
    [0, 9, 0,  0, 0, 0,  7, 3, 0],
    [0, 0, 2,  0, 0, 9,  0, 6, 0],

    [2, 0, 0,  0, 0, 0,  4, 0, 9],
    [0, 0, 0,  0, 7, 0,  0, 0, 0],
    [6, 0, 9,  0, 0, 0,  0, 0, 1],

    [0, 8, 0,  4, 0, 0,  1, 0, 0],
    [0, 6, 3,  0, 0, 0,  0, 8, 0],
    [0, 0, 0,  6, 0, 8,  0, 0, 0]
]
```

The goal is to replace the 0s with acceptable numbers. How to obtain a solution?

In [4]:
# Data
input_grid = [  # 0 represents empty cells
    [0, 0, 0,  2, 0, 5,  0, 0, 0],
    [0, 9, 0,  0, 0, 0,  7, 3, 0],
    [0, 0, 2,  0, 0, 9,  0, 6, 0],

    [2, 0, 0,  0, 0, 0,  4, 0, 9],
    [0, 0, 0,  0, 7, 0,  0, 0, 0],
    [6, 0, 9,  0, 0, 0,  0, 0, 1],

    [0, 8, 0,  4, 0, 0,  1, 0, 0],
    [0, 6, 3,  0, 0, 0,  0, 8, 0],
    [0, 0, 0,  6, 0, 8,  0, 0, 0]]
given = np.array(input_grid)  # numpy array for easy indexing

# Decision Variables
grid = cp.intvar(1, 9, shape=given.shape)  # Each cell can have a value between 1 and 9

# Model
model = cp.Model()

# Fix the non-zero cells to the given values
model.add(grid[given != 0] == given[given != 0])  # numpy's indexing

# No duplicate number in a row, for all rows
model.add([cp.AllDifferent(row) for row in grid])
# No duplicate number in a column, for all columns
model.add([cp.AllDifferent(col) for col in grid.T])  # numpy's transpose

# No duplicate number in a block, for all blocks
for i in range(0, 9, 3):
    for j in range(0, 9, 3):
        model.add(cp.AllDifferent(grid[i:i+3, j:j+3]))

# For block constraints: Alternative way with list comprehension
model.add([cp.AllDifferent(grid[i:i+3, j:j+3]) for i in range(0, 9, 3) for j in range(0, 9, 3)])

# Solve
model.solve()

# Print solution
print(grid.value())

[[3 7 8 2 6 5 9 1 4]
 [5 9 6 8 1 4 7 3 2]
 [1 4 2 7 3 9 5 6 8]
 [2 1 7 3 8 6 4 5 9]
 [8 5 4 9 7 1 6 2 3]
 [6 3 9 5 4 2 8 7 1]
 [7 8 5 4 2 3 1 9 6]
 [4 6 3 1 9 7 2 8 5]
 [9 2 1 6 5 8 3 4 7]]


### **Part 2: Exercises**

#### **1. Five Floors Problem**

Baker, Cooper, Fletcher, Miller, and Smith live on the first five floors of an apartment house.
Baker does not live on the fifth floor.
Cooper does not live on the first floor.
Fletcher does not live on either the fifth or the first floor.
Miller lives on a higher floor than does Cooper.
Smith does not live on a floor adjacent to Fletcher's.
Fletcher does not live on a floor adjacent to Cooper's.
They all live on different floors.
Find the floors where these people live.

In [5]:
# Decision Variables
lb = 1  # lower bound
ub = 5  # upper bound

B = cp.intvar(lb, ub)  # Baker
C = cp.intvar(lb, ub)  # Cooper
F = cp.intvar(lb, ub)  # Fletcher
M = cp.intvar(lb, ub)  # Miller
S = cp.intvar(lb, ub)  # Smith

# Since all the decision variables have the same domain, we can also use a more convenient way to define them:
B, C, F, M, S = floors = cp.intvar(lb, ub, shape=5)

# Model
model = cp.Model()

# Constraints
model.add(B != 5)  # Baker does not live on the fifth floor
model.add(C != 1)  # Cooper does not live on the first floor
model.add((F != 5) & (F != 1))  # Fletcher does not live on either the fifth or the first floor
model.add(M > C)  # Miller lives on a higher floor than does Cooper
model.add(cp.abs(S - F) != 1)  # Smith does not live on a floor adjacent to Fletcher
model.add(cp.abs(F - C) != 1)  # Fletcher does not live on a floor adjacent to Cooper
model.add(cp.AllDifferent([B, C, F, M, S]))  # They all live on different floors

# Solve
model.solve()

# Print solution
print(f'Baker lives on floor {B.value()}')
print(f'Cooper lives on floor {C.value()}')
print(f'Fletcher lives on floor {F.value()}')
print(f'Miller lives on floor {M.value()}')
print(f'Smith lives on floor {S.value()}')

Baker lives on floor 3
Cooper lives on floor 2
Fletcher lives on floor 4
Miller lives on floor 5
Smith lives on floor 1


#### **2. Bank Card**

My bank card has a 4 digit PIN, abcd. I use the following facts to help me remember it:

- no two digits are the same
- the 2-digit number cd is 3 times the 2-digit number ab
- the 2-digit number da is 2 times the 2-digit number bc

What is my PIN?

In [6]:
# Decision Variables
lb, ub = 0, 9  # lower bound and upper bound
a, b, c, d = digits = cp.intvar(lb, ub, shape=4)  # The four digits of the PIN

# Model
model = cp.Model()

# Constraints

model.add(cp.AllDifferent([a, b, c, d]))  # no two digits are the same
model.add(10 * c + d == 3 * (10 * a + b))  # cd is 3 times ab
model.add(10 * d + a == 2 * (10 * b + c))  # da is 2 times bc

# Solve
model.solve()

# Print solution
print(f"PIN: {a.value()}{b.value()}{c.value()}{d.value()}")

PIN: 2163


#### **3. Coin Change**

Alice needs to give Bob exactly 1.99 euros in change. She has six different types of coins with the following values: 1, 2, 5, 10, 25, and 50 cents. However, she only has a limited number of each coin type: 20 coins of 1 cent, 10 coins of 2 cents, 15 coins of 5 cents, 8 coins of 10 cents, 4 coins of 25 cents, and 2 coins of 50 cents.

How can Alice give Bob the exact change using the **fewest number of coins possible** while respecting the availability of each coin type?

In [7]:
# Parameters of the problem
amount = 199  # in cents
coin_types = [1, 2, 5, 10, 25, 50]
available_coins = [20, 10, 15, 8, 4, 2]
num_coins = len(coin_types)

# Decision Variables
coin_counts = cp.intvar(0, max(available_coins), shape=num_coins)  # number of coins of each type to give to Bob

# Model
model = cp.Model()

# Constraints

# The sum of the coins given to Bob must be equal to the amount of money to give him
model.add(sum(coin_counts[i] * coin_types[i] for i in range(num_coins)) == amount)

# The number of each type of coin given to Bob must not exceed the available coins
for i in range(num_coins):
    model.add(coin_counts[i] <= available_coins[i])

# Objective: Minimize the total number of coins given to Bob
model.minimize(sum(coin_counts))

# Solve
model.solve()

# Print solution
for i in range(num_coins):
    print(f"Number of {coin_types[i]} cent(s) coins given to Bob: {coin_counts[i].value()}")

Number of 1 cent(s) coins given to Bob: 0
Number of 2 cent(s) coins given to Bob: 2
Number of 5 cent(s) coins given to Bob: 0
Number of 10 cent(s) coins given to Bob: 2
Number of 25 cent(s) coins given to Bob: 3
Number of 50 cent(s) coins given to Bob: 2


#### **4. Magic Square**

A magic square is an $n \times n$ grid such that each cell contains a different integer from 1 to $n^2$ and the sum of the integers in each row, column and diagonal is equal.
Find a magic square for any size $n$, knowing that the sum of each row, column and diagonal has to be equal to $n(n^2 + 1)/2$ (an integer: div
operator). Remember there is no magic square for size 2.

<details>
  <summary>Click to reveal hint</summary>
  
```
"""
An example to show how to use indexing for diagonals in matrices.

Suppose that we have a grid of integers, and we require every number in the main
diagonal to be different, and also every number in the secondary diagonal to be
different.
"""

# Parameters
n = 50

# Decision Variables
grid = cp.intvar(0, n, shape=(n, n))

# Constraints
model = cp.Model()

model += cp.AllDifferent(grid[i, i] for i in range(n))  # Every number in the main diagonal is different
model += cp.AllDifferent(grid[i, n - 1 - i] for i in range(50))  # Every number in the secondary diagonal is different

# Solve
model.solve()
```
  
</details>

In [8]:
def magic_square(n):

  assert n != 2, "Magic square is not defined for size 2"

  magic_sum = n * (n**2 + 1) // 2  # sum of each row, column and diagonal

  # Decision Variables
  square = cp.intvar(1, n ** 2, shape=(n, n))  # the magic square

  # Model
  model = cp.Model()

  # Constraints

  # All numbers in the magic square must be different
  model.add(cp.AllDifferent(square))

  # The sum of the numbers in each row must be equal to the magic sum
  for i in range(n):
    model.add(sum(square[i, :]) == magic_sum)

  # The sum of the numbers in each column must be equal to the magic sum
  for j in range(n):
    model.add(sum(square[:, j]) == magic_sum)

  # The sum of the numbers in the main diagonal must be equal to the magic sum
  model.add(sum(square[i, i] for i in range(n)) == magic_sum)

  # The sum of the numbers in the other diagonal must be equal to the magic sum
  model.add(sum(square[i, n - 1 - i] for i in range(n)) == magic_sum)

  # Solve
  model.solve()

  # Print solution
  for i in range(n):
    for j in range(n):
      print(f"{square[i, j].value():2}", end=" ")
    print()


# Test
magic_square(5)

25 12  1  3 24 
 2 16 20 14 13 
23  6  9 22  5 
 7 10 18 11 19 
 8 21 17 15  4 


#### **5. Movie Scheduling**:

Consider the following scheduling problem. Imagine you are a highly-in-
demand actor, who has been presented with offers to star in n different movie
projects under development. Each offer comes specified with the first and last day of filming. To take the job, you must commit to being available throughout this entire period. Thus, you cannot simultaneously accept two jobs whose intervals overlap.
For an artist such as yourself, the criteria for job acceptance is clear: you want to make as much money as possible. Because each of these films pays the same fee per film, this implies you seek the largest possible set of jobs (intervals) such that no two of them conflict with each other.

Here is the list of movies along with their first and last day of filming:
```
movies = [  # title, start, end
    ["Tarjan of the Jungle", 4, 13],
    ["The Four Volume Problem", 17, 27],
    ["The President's Algorist", 1, 10],
    ["Steiner's Tree", 12, 18],
    ["Process Terminated", 23, 30],
    ["Halting State", 9, 16],
    ["Programming Challenges", 19, 25],
    ["Discrete Mathematics", 2, 7],
    ["Calculated Bets", 26, 31]
]
```

Which movies would you choose to maximize your income?

In [9]:
# Data
movies = [  # title, start, end
    ["Tarjan of the Jungle", 4, 13],
    ["The Four Volume Problem", 17, 27],
    ["The President's Algorist", 1, 10],
    ["Steiner's Tree", 12, 18],
    ["Process Terminated", 23, 30],
    ["Halting State", 9, 16],
    ["Programming Challenges", 19, 25],
    ["Discrete Mathematics", 2, 7],
    ["Calculated Bets", 26, 31]
]
num_movies = len(movies)

# Decision Variables
selected_movies = cp.boolvar(shape=num_movies)  # 1 if the movie is selected, 0 otherwise

# Model
model = cp.Model()

# Constraints

# Add constraint for non-overlapping movie schedules
for i in range(num_movies):
    for j in range(num_movies):
        # Check if the intervals overlap for each pair of movies
        if (i != j  # Different movies
                and movies[i][2] > movies[j][1]  # Movie i ends after movie j starts
                and movies[j][2] > movies[i][1]  # Movie j ends after movie i starts
        ):
            # Then, the movies cannot be selected together
            model.add(selected_movies[i] + selected_movies[j] <= 1)


# Objective: Maximize the number of selected movies
model.maximize(sum(selected_movies))

# Solve
model.solve()

# Print solution
print("Selected movies:\n")
for i in range(num_movies):
    if selected_movies[i].value():
        print(movies[i][0])

Selected movies:

Halting State
Programming Challenges
Discrete Mathematics
Calculated Bets


#### **6. Robbery**

Following a robbery, an inspector interviewed six suspects. The getaway car had been barely big enough to hold two, so she reckoned that at least four of them were innocent. She also supposed that the innocent ones would tell the truth, while the guilty one or ones would lie. What they actually said was:
- ARTIE: "It wasn't me."
- BILL: "Crackitt was in it up to his neck."
- CRACKITT: "No I wasn't."
- DODGY: "If Crackitt did it, Bill did it with him."
- EDGY: "Nobody did it alone."
- FINGERS: "It was Artie and Dodgy together."

Who is guilty?

<details>
  <summary>Click to reveal hint</summary>

  **Logic Truth Table:**
  
  | p     | q     | ¬p   | p ∧ q | p ∨ q | p → q | p ↔ q |
  |-------|-------|------|-------|-------|-------|-------|
  | True  | True  | False| True  | True  | True  | True  |
  | True  | False | False| False | True  | False | False |
  | False | True  | True | False | True  | True  | False |
  | False | False | True | False | False | True  | True  |

  **Explanation of Symbols:**
  - ¬p: Negation (NOT p)
  - p ∧ q: Conjunction (AND)
  - p ∨ q: Disjunction (OR)
  - p → q: Implication (If p then q)
  - p ↔ q: Biconditional (p if and only if q)

</details>

In [10]:
# Decision Variables for each suspect representing if they are guilty
artie = cp.boolvar(name="Artie")
bill = cp.boolvar(name="Bill")
crackitt = cp.boolvar(name="Crackitt")
dodgy = cp.boolvar(name="Dodgy")
edgy = cp.boolvar(name="Edgy")
fingers = cp.boolvar(name="Fingers")
suspects = [artie, bill, crackitt, dodgy, edgy, fingers]

# Model
model = cp.Model()

# Constraints

# At most two are guilty because the getaway car was small
model.add(sum(suspects) <= 2)

# Statement Constraints; if the suspect is guilty, they are lying, so their statement is false

# Artie: "It wasn't me."
artie_statement = ~artie
model.add(artie == ~artie_statement)

# Bill: "Crackitt was in it up to his neck."
bill_statement = crackitt
model.add(bill == ~bill_statement)

# Crackitt: "No I wasn't."
crackitt_statement = ~crackitt
model.add(crackitt == ~crackitt_statement)

# Dodgy: "If Crackitt did it, Bill did it with him."
dodgy_statement = crackitt.implies(bill)
model.add(dodgy == ~dodgy_statement)

# Edgy: "Nobody did it alone."
edgy_statement = sum(suspects) > 1
model.add(edgy == ~edgy_statement)

# Fingers: "That’s right: it was Artie and Dodgy together."
fingers_statement = artie & dodgy
model.add(fingers == ~fingers_statement)

# Solve
model.solve()

# Print solution
print("The guilty suspect(s) is/are:")
for i in range(6):
    if suspects[i].value():
        print(f"{suspects[i].name}")

The guilty suspect(s) is/are:
Bill
Fingers


#### **Extra Exercises**

> The following exercises are **extra practice** problems. They will **not be covered** during the regular exercise sessions. However, they offer a great opportunity to practice more. You are encouraged to try them at home to challenge yourself further!


##### **Extra 1: Send More Money**

The "Send More Money" puzzle is a classic cryptarithmetic problem where each letter represents a unique digit. The goal is to assign digits to the letters such that the following equation holds true:
```
   SEND
 + MORE
 ------
  MONEY
```
Each letter must be assigned a unique digit from 0 to 9, and the first letter of each word cannot be zero. What is the digit assigned to each letter?

In [11]:
# Decision Variables
s, e, n, d, m, o, r, y = cp.intvar(0, 9, shape=8)

# Model
model = cp.Model(
    cp.AllDifferent([s, e, n, d, m, o, r, y]),

    (sum([s, e, n, d] * np.array([1000, 100, 10, 1])) \
     + sum([m, o, r, e] * np.array([1000, 100, 10, 1])) \
     == sum([m, o, n, e, y] * np.array([10000, 1000, 100, 10, 1]))),

    s > 0,
    m > 0,
)

# Solve
model.solve()

# Print solution
for letter, digit in zip("sendmory", [s.value(), e.value(), n.value(), d.value(), m.value(), o.value(), r.value(), y.value()]):
    print(f"{letter}: {digit}")

s: 9
e: 5
n: 6
d: 7
m: 1
o: 0
r: 8
y: 2


##### **Extra 2: Minesweeper**

Numbers and mines are randomly distributed in a given board.
The number on a cell shows the exact number of mines around this cell in all 8 directions: top, top-right, right, bottom-right, bottom, bottom-left, left, top-left.

The task is to determine which cells contain mines, the initial board is given below.

<details>
  <summary>Click to reveal hint</summary>

```
"""
An example to show how to find neighbors (top, right, bottom, left) of a cell in a grid.
"""

n = 4  # 4x4 grid
grid = cp.boolvar(shape=(n, n))

model = cp.Model()

for i in range(n):
    for j in range(n):
        # Collect neighbors for each cell
        neighbors = []
        for a, b in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            if 0 <= i + a < n and 0 <= j + b < n:
                neighbors.append(grid[i + a, j + b])

        print(f"Neighbors of cell {grid[i, j]}: {neighbors}")

print("The grid:")
print(grid)
```

</details>

In [12]:
# Data
X = -1
game = np.array([  # 0-8: number of mines around, -1: not opened
    [2, 3, X, 2, 2, X, 2, 1],
    [X, X, 4, X, X, 4, X, 2],
    [X, X, X, X, X, X, 4, X],
    [X, 5, X, 6, X, X, X, 2],
    [2, X, X, X, 5, 5, X, 2],
    [1, 3, 4, X, X, X, 4, X],
    [0, 1, X, 4, X, X, X, 3],
    [0, 1, 2, X, 2, 3, X, 2]
])
rows, cols = game.shape
S = [-1, 0, 1]  # for the neighbors of a cell


# Decision Variables
mines = cp.boolvar(shape=game.shape)  # True: mine, False: not mine

# Model
model = cp.Model()
for (r, c), val in np.ndenumerate(game):
    if val != X:
        # This cell cannot be a mine
        model.add(mines[r, c] == 0)
        # Count neighbors
        model.add(sum(mines[r + a, c + b]
                      for a in S for b in S
                      if 0 <= r + a < rows and 0 <= c + b < cols and (a, b) != (0, 0))
                  == val)

# Solve
model.solve()

# Print solution
for i in range(rows):
    for j in range(cols):
        print(f"{int(mines[i, j].value())}", end=" ")
    print()

0 0 1 0 0 1 0 0 
1 1 0 0 1 0 1 0 
0 0 1 1 0 1 0 1 
1 0 1 0 1 1 0 0 
0 1 1 1 0 0 1 0 
0 0 0 0 1 1 0 1 
0 0 1 0 1 0 1 0 
0 0 0 1 0 0 1 0 
